In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Khmer next word prediction (unsegmented)

Model: mT5-small
Task: Predict the NEXT word-like span only
Metric: Loss + Perplexity

---

# Library

In [ ]:
!pip install -q transformers datasets sentencepiece accelerate

import re
import math
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    MT5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback)

# Config

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/dataset_for_spellcheck"
FILES = ["kh_oscars_Dataset.txt", "​kh_CC100.txt"]

MODEL_NAME = "google/mt5-small"
OUTPUT_DIR = "./khmer_nextword_mt5"

TEST_LINES = 4000   # None for full dataset
EPOCHS = 3
BATCH_SIZE = 16
LR = 2e-4

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# STEP 1: LOAD & CLEAN UNSEGMENTED KHMER

In [ ]:
lines = []
for fname in FILES:
    with open(f"{DATASET_PATH}/{fname}", "r", encoding="utf-8") as f:
        file_lines = f.readlines()
        lines.extend(file_lines)
    print(f"Loaded {len(file_lines)} lines from {fname}")

def clean_khmer(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(
        r"[^\u1780-\u17FF\u19E0-\u19FF\s.,!?;:'\"()\-ៗ់៎៏័៍៑្]",
        "",
        text
    )
    return text.strip()

cleaned = [clean_khmer(l) for l in lines if len(clean_khmer(l)) > 12]
print(f" Cleaned lines kept: {len(cleaned)}")

# STEP 2: CREATE NEXT WORD PAIRS (UNSEGMENTED)

In [ ]:
def create_next_word_pairs(
    lines,
    min_prefix=8,
    max_prefix=48,
    target_chars=6,   # ~one Khmer word
    stride=2,
    max_lines=None
):
    pairs = []

    if max_lines:
        lines = lines[:max_lines]

    for text in lines:
        L = len(text)
        if L < min_prefix + target_chars:
            continue

        for p in range(min_prefix, min(L - target_chars, max_prefix), stride):
            input_text = text[:p]
            target_text = text[p:p + target_chars]

            pairs.append({
                "input_text": input_text,
                "target_text": target_text
            })

    return pairs

pairs = create_next_word_pairs(cleaned, max_lines=TEST_LINES)
np.random.shuffle(pairs)

print(f"✓ Total next-word samples: {len(pairs)}")

# STEP 3: DATASET SPLIT

In [ ]:
dataset = Dataset.from_list(pairs)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_ds = dataset["train"]
val_ds = dataset["test"]

print(f"✓ Train: {len(train_ds)} | Val: {len(val_ds)}")

# STEP 4: MODEL & TOKENIZER

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)

model.gradient_checkpointing_enable()
model.config.use_cache = False

# STEP 5: TOKENIZATION

In [ ]:
def preprocess(batch):
    inputs = ["next: " + t for t in batch["input_text"]]

    model_inputs = tokenizer(
        inputs,
        max_length=64,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target_text"],
            max_length=8,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = [
        [t if t != tokenizer.pad_token_id else -100 for t in label]
        for label in labels["input_ids"]
    ]

    return model_inputs

train_tok = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_tok = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)


# STEP 6: PERPLEXITY CALLBACK

In [ ]:
class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_loss" in metrics:
            ppl = math.exp(metrics["eval_loss"])
            metrics["perplexity"] = ppl
            print(f"📊 Perplexity: {ppl:.4f}")

# STEP 7: TRAINING CONFIG

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=32,
    learning_rate=LR,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    fp16=False,
    report_to="none",
    predict_with_generate=False  # IMPORTANT: no auto-suggestion
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model)
)

trainer.add_callback(PerplexityCallback())

In [ ]:
# STEP 8: TRAIN

print("\n🚀 Training Khmer Next Word Model...")
trainer.train()

# STEP 9: SAVE

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# STEP 10: SIMPLE NEXT WORD TEST (SINGLE OUTPUT)

print("\n=== NEXT WORD PREDICTION TEST ===")

model.eval()

test_inputs = [
    "សម្តេចពុក",
    "ការអប់រំ",
    "ប្រទេសកម្ពុជា"
]

for text in test_inputs:
    inputs = tokenizer(
        "next: " + text,
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=16,
            num_beams=1
        )

    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"\nInput: {text}")
    print(f"Next word: {prediction}")

print("\n✅ Khmer Next Word Prediction Ready")



=== STEP 1: Loading Data ===
✓ Loaded 147208 lines from kh_oscars_Dataset.txt
✓ Loaded 3012632 lines from ​kh_CC100.txt
✓ Cleaned lines kept: 3087920

=== STEP 2: Creating Next Word Pairs ===
✓ Total next-word samples: 79998
✓ Train: 71998 | Val: 8000

=== STEP 4: Loading Model ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pytho

Map:   0%|          | 0/71998 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

...


🚀 Training Khmer Next Word Model...


Step,Training Loss,Validation Loss
500,4.317300,3.717050
1000,3.864300,3.474234
1500,3.639900,3.248202
2000,3.519700,3.133784
2500,3.419000,3.041406
3000,3.312200,2.919609
3500,3.222800,2.850423
4000,3.092000,2.766450
4500,3.066300,2.717021
5000,2.933000,2.660603


📊 Perplexity: 41.1428
📊 Perplexity: 32.2731
📊 Perplexity: 25.7440
📊 Perplexity: 22.9607
📊 Perplexity: 20.9346
📊 Perplexity: 18.5340
📊 Perplexity: 17.2951
📊 Perplexity: 15.9021
📊 Perplexity: 15.1352
📊 Perplexity: 14.3049
📊 Perplexity: 13.7650
📊 Perplexity: 13.2488
📊 Perplexity: 12.9884
📊 Perplexity: 12.3608
📊 Perplexity: 12.0608
📊 Perplexity: 11.6556
📊 Perplexity: 11.3446
📊 Perplexity: 11.2039
📊 Perplexity: 10.9360
📊 Perplexity: 10.7721
📊 Perplexity: 10.6699
📊 Perplexity: 10.5274


Step,Training Loss,Validation Loss
500,4.317300,3.717050
1000,3.864300,3.474234
1500,3.639900,3.248202
2000,3.519700,3.133784
2500,3.419000,3.041406
3000,3.312200,2.919609
3500,3.222800,2.850423
4000,3.092000,2.766450
4500,3.066300,2.717021
5000,2.933000,2.660603


📊 Perplexity: 10.4069
📊 Perplexity: 10.3584
📊 Perplexity: 10.3467
📊 Perplexity: 10.2559
📊 Perplexity: 10.2532

=== NEXT WORD PREDICTION TEST ===

Input: សម្តេចពុក
Next word: ាភិបាល

Input: ការអប់រំ
Next word: អប់រំ

Input: ប្រទេសកម្ពុជា
Next word: ជប៉ុន

✅ Khmer Next Word Prediction Ready


In [ ]:
!cp -r /content/khmer_nextword_mt5/checkpoint-13500 /content/drive/MyDrive/MT5_Models/

In [ ]:
model.eval()

test_inputs = [
    "សាលា",
    "សុថាយស្មោះ",
    "ប្រទេសចោរ"
]

for text in test_inputs:
    inputs = tokenizer(
        "next: " + text,
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=25,
            num_beams=1
        )

    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"\nInput: {text}")
    print(f"Next word: {prediction}")

print("\n✅ Khmer Next Word Prediction Ready")


Input: សាលា
Next word: អប់រំ

Input: សុថាយស្មោះ
Next word: សម្រាប

Input: ប្រទេសចោរ
Next word: លួចច្

✅ Khmer Next Word Prediction Ready


In [ ]:
!cp -r /content/khmer_nextword_mt5/checkpoint-13500 /content/drive/MyDrive/MT5_Checkpoints/

In [ ]:
from safetensors.torch import load_file

# Step 2: Load safetensors weights
checkpoint_path = "/content/khmer_nextword_mt5/checkpoint-13500/model.safetensors"
state_dict = load_file(checkpoint_path)

# Step 3: Save as .pth file
torch.save(state_dict, "/content/khmer_nextword_mt5/khmer_nextword_mt5.pth")

# Step 4: Copy the .pth file into Google Drive
!cp /content/khmer_nextword_mt5/khmer_nextword_mt5.pth /content/drive/MyDrive/